In [1]:
import llm_core.llm as L
import os

In [ ]:
# reduce available words to see difference in grammar
# n-5 vocab

# better benchmarking rules
# LLM as judge to an extent (multiple?)
# grade out of 100 with rubric
# do by hand too

In [ ]:
#os.environ['MODEL_PATH'] = "/data/ai_club/llms/qwen2-7b-instruct-q5_k_m.gguf"
LLM_GLOBAL_INSTANCE = L.Llama(
                n_ctx=1000,
                model_path=os.environ['MODEL_PATH'],
                n_gpu_layers=-1, verbose=0,
                # embedding=True
            )

In [3]:
# system_input = "You are a Japanese language assistant who speaks in consice, simple sentences."
# user_input = "Tell me about your favorite Kanji, in Japanese using Kanji, Hiragana, and Katakana when appropriate"
system_input = "You are a helpful assistant. Respond with prompts suitable for Japanese children's books, following the [word] が [word] です form."
user_input = "赤色は何をしてるですか？"

In [ ]:
# No Kanji usage, rare katakana usage
grammar_string = r"""
root ::= sentence{1,2}
allowedchars ::= [\u3040-\u30FF, \u4e00 - \u9faf]
word ::= allowedchars
sentence ::= word particle word desu end
particle ::= "が"
desu ::= "です"
end ::= "。"
"""
my_grammar = L.LlamaGrammar.from_string(grammar_string)

LLM_GLOBAL_INSTANCE.create_chat_completion([
        {"role": "system", "content": system_input},
        {"role": "user", "content": user_input}
    ], 
    temperature=0.1,
    max_tokens=50,
    grammar=my_grammar
    )

{'id': 'chatcmpl-a0d28538-0d15-4bbc-babe-8b2bc0067abc',
 'object': 'chat.completion',
 'created': 1740143766,
 'model': '/data/ai_club/llms/qwen2.5-7b-instruct-q8_0-00001-of-00003.gguf',
 'choices': [{'index': 0,
   'message': {'role': 'assistant', 'content': 'あがめです。'},
   'logprobs': None,
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 50, 'completion_tokens': 5, 'total_tokens': 55}}

In [ ]:
#With basic prompt, no character restrictions spamps \u0000 or \uffff
#Good responses with user_input = "Tell me about your favorite Kanji, in Japanese using Kanji, Hiragana, and Katakana when appropriate"
#Talks in any vocab, but keeps sentences in が + です format. :D
# Try telling it to write with spaces between words?
grammar_string = r"""
root ::= sentence{1,2}
allowedchars ::= [\u4e00 - \u9faf, \u3000-\u303f, \u3040-\u309F, \u30a0-\u30ff]
word ::= (unicodechar)+
unicodechar ::= [\u0000-\uffff]
sentence ::= word particle word desu end
particle ::= "が"
desu ::= "です"
end ::= "。"
"""
my_grammar = L.LlamaGrammar.from_string(grammar_string)

LLM_GLOBAL_INSTANCE.create_chat_completion([
        {"role": "system", "content": system_input},
        {"role": "user", "content": user_input}
    ], 
    temperature=0.1,
    max_tokens=50,
    grammar=my_grammar
    )

#Really good example sentence: '赤色は炎のように光ってます が 楽しく感じます です。'

{'id': 'chatcmpl-77bc66c8-bc91-4339-b8e4-fc5140beaa4a',
 'object': 'chat.completion',
 'created': 1740143768,
 'model': '/data/ai_club/llms/qwen2.5-7b-instruct-q8_0-00001-of-00003.gguf',
 'choices': [{'index': 0,
   'message': {'role': 'assistant', 'content': '赤色は炎のように光ってます が 楽しく感じます です。'},
   'logprobs': None,
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 50, 'completion_tokens': 19, 'total_tokens': 69}}

In [7]:
LLM_GLOBAL_INSTANCE.reset()
system_input = "You are a helpful assistant. Respond with prompts suitable for Japanese children's books, following the [word] が [word] です form."
user_input = "赤色は何をしてるですか？"

In [8]:
#only 1 or 2 allowedchar makes for much shorter+concise sentences
#adding more sentences makes them incoherent
grammar_string = r"""
root ::= sentence
allowedchars ::= [\u0000-\uffff]
word ::= allowedchars
sentence ::= word particle word desu end
particle ::= "が"
desu ::= "です"
end ::= "。"
"""
my_grammar = L.LlamaGrammar.from_string(grammar_string)

#grammar works better with Japanese prompts
#English prompts are better without grammar
#Not much variety between responses though
#How does Llama grammar work??!!? Logits?

In [ ]:
training_data = list()
training_size = 10
for i in range(training_size):
    response = LLM_GLOBAL_INSTANCE.create_chat_completion([
        {"role": "system", "content": system_input},
        {"role": "user", "content": user_input}
    ], 
    temperature=0.1,
    max_tokens=50,
    grammar=my_grammar
    )
    training_data.append(response['choices'][0]["message"]["content"])
training_data

['赤が喜です。',
 '赤が喜です。',
 '赤が火です。',
 '赤が喜です。',
 '赤が力です。',
 '赤が力です。',
 '赤が怒です。',
 '赤が喜です。',
 '赤が怒です。',
 '赤が光です。']

In [10]:
# i and na-adj don't work well
LLM_GLOBAL_INSTANCE.reset()
system_input = "You are a helpful assistant. Respond with prompts suitable for Japanese children's books, using adjectives to describe in the [word] が [adjective] form."
user_input = "太陽の景色を描いてください"
grammar_string = r"""
root ::= sentence
allowedchars ::= [\u0000-\uffff]
word ::= allowedchars
sentence ::= word particle adjective shii end
particle ::= "が"
adjective ::= [\u0000-\uffff]
shii ::= "しい"
desu ::= "です"
end ::= "。"
"""
my_grammar = L.LlamaGrammar.from_string(grammar_string)

In [ ]:
training_data = list()
training_size = 10
for i in range(training_size):
    response = LLM_GLOBAL_INSTANCE.create_chat_completion([
        {"role": "system", "content": system_input},
        {"role": "user", "content": user_input}
    ], 
    temperature=0.1,
    max_tokens=50,
    grammar=my_grammar
    )
    training_data.append(response['choices'][0]["message"]["content"])
training_data

['太が出しい。',
 '太が出しい。',
 '太が出しい。',
 '太が出しい。',
 '太が輝しい。',
 '太があしい。',
 '太が出しい。',
 '太があしい。',
 '太が出しい。',
 '太が出しい。']

In [ ]:
# Needs a good system prompt to use しか at all
# Difficult to get it to use しか normally and not just repeat the user question with it.
LLM_GLOBAL_INSTANCE.reset()
system_input = "You are a helpful assistant. Respond with prompts using the しか～ない Japanese grammar pattern"
user_input = "あんたは普通な人って想像してください。。何をできますか？"
grammar_string = r"""
root ::= sentence
allowedchars ::= [\u0000-\uffff]
word ::= allowedchars+
sentence ::= word particle word end
particle ::= "しか"
end ::= "。"
"""
my_grammar = L.LlamaGrammar.from_string(grammar_string)

In [26]:
training_data = list()
training_size = 10
for i in range(training_size):
    response = LLM_GLOBAL_INSTANCE.create_chat_completion([
        {"role": "system", "content": system_input},
        {"role": "user", "content": user_input}
    ],
    temperature=0.1,
    max_tokens=50,
    grammar=my_grammar
    )
    training_data.append(response['choices'][0]["message"]["content"])
training_data

['あなたは普通の人間しか想像できないでしょう。何をできますか？？この質問には、何か特定のスキルや能力があるわけではないので、日常生活の中で必要な基本的な能力が考えられます。例えば、料理を作ったり、掃除を',
 'あなたは普通な人って想像してみてください。何をできるか ila しか想像できませんね。',
 'あんたは普通な人って想像してください。何か特別なことをするしかないのですね。',
 'あんたは普通な人って想像してください。特別なスキルがある場合もあるが、ほとんどは料理するしかありませんな。それも時々しかできませんね。',
 'あなたは普通な人って想像してください。何か特別なことをできるしかありませんな。',
 'あんたは普通な人って想像してください。何か特別なスキルがあるなんて考えられませんか？もしそんなことがないなら、仕事中はメールチェックするしかありませんね。',
 '普通な人なら、料理を作ることはほとんどしかありません。何か特別な能力があるのかな？それとも仕事があるのかな？それ以外には何もできない気がするよ。',
 '普通の人は何か特別なスキルがなく、特殊な状況以外では何もできないという考えでは、少し狭いですね。しかり、基本的な日常生活のスキルしかありません。しかり、何かを始めるためのエネルギーと時間',
 'あんたは普通な人って想像してください。そんな人なら料理を作ることはほとんどありませんはずです。しかしこの一つだけはできるかもしれません。しかりあんたなら、お弁当を作るのは難しくないはずです。',
 'あんたは普通な人って想像してください。何をできるしかありませんな。']

In [ ]:
# Responses might be better with grammar? More english junk, but the non-junk are shorter and more straight-forward
LLM_GLOBAL_INSTANCE.reset()
system_input = "You are a helpful assistant. Respond with prompts suitable for Japanese children's books, using the かどうか grammar pattern"
user_input = "Ask me a quesiton in Japanese."
grammar_string = r"""
root ::= sentence
allowedchars ::= [\u0000-\uffff]
word ::= allowedchars+
sentence ::= word particle word end
particle ::= "かどうか"
end ::= "。"
"""
my_grammar = L.LlamaGrammar.from_string(grammar_string)

In [ ]:
training_data = list()
training_size = 10
for i in range(training_size):
    response = LLM_GLOBAL_INSTANCE.create_chat_completion([
        {"role": "system", "content": system_input},
        {"role": "user", "content": user_input}
    ], 
    temperature=0.1,
    max_tokens=50,
    grammar=my_grammar
    )
    training_data.append(response['choices'][0]["message"]["content"])
training_data

['日本のどの都市に住むかどうか聞かせてください。',
 '日本語でお聞きBecome a question using the かどうか grammar pattern. \n\nあなたは今日お絵かきをするかどうか知りたいです。',
 "日本語でお尋ねしますね。この質問には、「かどうか」を使用してみました。\n\n絵本の主人公は、新しい友達ができたかどうか知りたいですか？（ Picture book's protagonist wants to know if they made a new friend, かどうか?） \n\nこの質問に答えると、主人公の気持ちを理解できるかもしれませんね。",
 'あなたは今日公園に行くかどうか知っていますか？（あなたはきょうこうえんにい행을 가는かどうかしっていますか？） \n\nこの文は、今日公園にいくかどうかという質問を子供向けに簡潔に表現しています。"かどうか"は、質問の内容を明確にし、どちらの選択肢でも答えがあるということを示しています。',
 'あなたは今日学校に行きますか行かないか知りたいです。答え帮我填空吧，可以用 行きます or 行きません。例如：あなたは今日学校に行きます____。填空后句子的意思是“你今天去学校____。”。现在请你填空完成这个句子吧！（あなたは今日学校に行きます____。）填空选项：行きます or 行きません。请根据实际情况填写。例如，如果今天你去学校，就填写“行きます”。如果不去，就填写“行きません”。试着自己完成句子吧！如果你去学校，就填写“行きます”。如果你不去，就填写“行きません”。现在试试看吧！（あなたは今日学校に行きます____。）希望你喜欢这个游戏！如果你需要进一步的帮助，随时告诉我哦！接下来，我们再来一个类似的练习吧！你最喜欢的颜色是红色かどうか。填空后句子的意思是“你喜欢的颜色是红色____。”。请你填空完成这个句子。如果喜欢，就填写“はい”；如果不喜欢，就填写“いいえ”。例如：あなたは favorite_color は赤色____。填空选项：はい or いいえ。请根据实际情况填写。例如，如果你喜欢红色，就填写“はい”。如果你不喜欢，就填写“いいえ”。试着自己完成句子吧！（あなたは favorite_color は赤色____。）希望你喜欢这个游戏！如果你需要进一步的帮

In [59]:
import re
for i in range(len(training_data)):
    print(re.findall("r[A-Za-z]", training_data[i]))
    if (len(re.findall("r[A-Za-z]", training_data[i])) > 0):
        japanese = re.sub("r[A-Za-z]", "", training_data[i])
        training_data[i] = japanese
training_data

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


['日本の動物公园にいってseeingか？それとも家で絵本を読むadingをするか？どちらが好きですか？（你更喜欢去日本的动物园，还是喜欢待在家里读书呢？）\r\n\r\nこの質問は、あなたが動物公园で動物を見ることと、家で静かに本を読むことのどちらが好きなかを知るためのものです。どちらも楽しい経験だと思いますが、どちらがあなたの好みに合っているか教えてください。その答えを元に、次の本の冒頭を決めてみましょう。あなたが動物公园に行くことを選んだら、私たちが動物たちの冒険を書きます。それとも読書を選んだら、本の主人公が新しい絵本世界で冒険する様子を書きます。どちらも面白いストーリーになると思いますよ。どの選択が正しいかはあなた次第です。公園に行くかどうか、読書をするかどうか、どちらを選んでみましょう。',
 '日本のどの都市に住んでいるかどうか教えてください。',
 'あなたは今日公園に行きたいかどうか教えてください。',
 'あなたは TODAYの朝に公園に行きますか？（今日の朝に公園に行きますか？） \n\nこの文章は、あなたが今日の朝に公園に行くかどうか尋ねる質問になっています。',
 'あなたは TODAY の学校を不去にするかどうか知りたいです。',
 'あなたは TODAY の朝起きてから学校に行くかどうか 知っていますか？（あなたは今日の朝起きてから学校に行くかどうか知っていますか？） \n\nこの質問は、子どもたちが日常的な行動について考えさせるのに役立ちます。例えば、「はい、-knows-」と答えると、話題を学校への通学や朝の準備などに導くことができます。',
 'あなたは今日公園に行きますか？行きませんか？どちらですか？（あなたは今日公園に行きますか、行きませんか、どちらですか？） \n\nこの文は、あなたが公園に行こうかどうかを尋ねています。公園に行くかどうか迷っている場合や、知りたい場合は、はいまたはいいえで答えてください。',
 'あなたは今お気に入りのキャラクターが誰ですか？それがあなたの好きなキャラクターかどうか知りたいです。',
 'あなたは今日お絵かきをしたいかどうか知りたいです。',
 'あなたは kitty に会ったことがありますか？それともまだ会っていないことがありますか？（あなたは kitty に会ったことがありますか、それと

# Training Tests

In [60]:
import torch
import torch.nn as nn
from transformers.models.qwen2.modeling_qwen2 import Qwen2DecoderLayer
from transformers import Qwen2ForCausalLM, Qwen2Config
import transformers
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm
import gc

In [61]:
class IMDecoderLayer(nn.Module):
    mask = None
    vspace_to_emb = None
    emb_to_vspace = None
    post_ff_norm = None

    def __init__(self, original_layer, emb_to_vspace, config):
        super().__init__()
        self.original_layer = original_layer

        if IMDecoderLayer.mask == None:
            IMDecoderLayer.mask = torch.zeros(config.vocab_size).to('cuda')

        if IMDecoderLayer.vspace_to_emb == None:
            IMDecoderLayer.vspace_to_emb =  nn.Linear(config.vocab_size, config.hidden_size).to('cuda')
        
        if IMDecoderLayer.emb_to_vspace == None:
            IMDecoderLayer.emb_to_vspace =  emb_to_vspace

        if IMDecoderLayer.post_ff_norm == None:
            IMDecoderLayer.post_ff_norm = nn.RMSNorm(config.hidden_size).to('cuda')

    def forward(self, hidden_states, *args, **kwargs):
        hidden_states = self.original_layer(hidden_states, *args, **kwargs)
        hidden_states = hidden_states[0]

        # TODO: for inference, bypass this process on all tokens but the current one? Possible problem: maybe it will learn to rely on past tokens being allowed ones later in residual.
        # ^ maybe this process should only be done on last token if later note is a problem. If it isnt a problem, then it's preferable to do this b/c the layer will train faster.

        hidden_states = IMDecoderLayer.post_ff_norm(hidden_states)
        residual = hidden_states
        hidden_states = IMDecoderLayer.emb_to_vspace(residual)
        hidden_states *= IMDecoderLayer.mask
        hidden_states = IMDecoderLayer.vspace_to_emb(hidden_states)
        # hidden_states = (hidden_states + residual)/2 # NOTE: should also be normalization? -- avg: has to learn to recreate residual with affects added on -- add&norm: most things can be zero and only learns to affect some values
        hidden_states = hidden_states + residual # input layernorm means don't need to norm here? (and SHOULD norm after hidden state??)

        return (hidden_states,)

In [62]:
MODEL_NAME = 'Qwen/Qwen2.5-0.5B-Instruct'

config = Qwen2Config.from_pretrained(MODEL_NAME)

model = Qwen2ForCausalLM.from_pretrained(MODEL_NAME).to('cuda')

# FREEZE existing model. Only the new layer in IMDecoderLayer will be trained
for param in model.parameters():
    param.requires_grad = False

tokenizer = transformers.AutoTokenizer.from_pretrained(MODEL_NAME)

for i, _ in enumerate(model.model.layers):
    model.model.layers[i] = IMDecoderLayer(model.model.layers[i], model.lm_head, config)

print(model)

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x IMDecoderLayer(
        (original_layer): Qwen2DecoderLayer(
          (self_attn): Qwen2Attention(
            (q_proj): Linear(in_features=896, out_features=896, bias=True)
            (k_proj): Linear(in_features=896, out_features=128, bias=True)
            (v_proj): Linear(in_features=896, out_features=128, bias=True)
            (o_proj): Linear(in_features=896, out_features=896, bias=False)
          )
          (mlp): Qwen2MLP(
            (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
            (up_proj): Linear(in_features=896, out_features=4864, bias=False)
            (down_proj): Linear(in_features=4864, out_features=896, bias=False)
            (act_fn): SiLU()
          )
          (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
          (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        )
      )
   

In [63]:
EPOCHS = 10
BATCH_SIZE = 1
optimizer = torch.optim.AdamW(
    [p for p in IMDecoderLayer.vspace_to_emb.parameters() if p.requires_grad]+
    [p for p in IMDecoderLayer.post_ff_norm.parameters() if p.requires_grad],
    lr=5e-5
)
dataloader = torch.utils.data.DataLoader(training_data, batch_size=BATCH_SIZE, shuffle=True)

In [64]:
losses = []

for epoch in range(EPOCHS):
    i = 0
    for batch in (pbar := tqdm(dataloader)):
        try:
            optimizer.zero_grad()

            tokens = tokenizer(batch, return_tensors='pt', padding=True)
            tokens = {k:v.to('cuda') for k,v in tokens.items()}

            # set mask from batch
            IMDecoderLayer.mask *= 0
            IMDecoderLayer.mask[tokens['input_ids'].unique()] += 1
            
            outputs = model(**tokens, labels=tokens['input_ids'])
            loss = outputs.loss

            loss.backward()
            optimizer.step()

            loss_avg = float(loss)/BATCH_SIZE
            losses.append(loss_avg)
            pbar.set_postfix(loss=loss_avg)

        except torch.OutOfMemoryError:
            print(f'OOM on {i}')
            try: del tokens, outputs
            except NameError: pass
            gc.collect()
        i += 1

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

In [65]:
prompt = "USER: 僕に「かどうか」って質問を聞いてください。 \nASSISTANT: "

In [66]:
tokens = tokenizer([prompt], return_tensors='pt', padding=True)
tokens = {k:v.to('cuda') for k,v in tokens.items()}

out = model.generate(
# out = model(
    **tokens,
    # labels=tokens['input_ids'],
    max_new_tokens=50,
    pad_token_id=tokenizer.eos_token_id,
    temperature=0.001,
    do_sample=True,
    return_dict_in_generate=True,
    output_hidden_states=True
)

out_ids = out['sequences']
print(tokenizer.decode(out_ids[0]))

USER: 僕に「かどうか」って質問を問いてください。 
ASSISTANT: ご質なこといません。この質の助けは、助けです。この助けを求められます。当助けが理想的助けです。この助けを求められます。当助けを求められます。当助けを求められます。当助けを求められます。当助けを求め


In [67]:
def output_w_mask(prompt, mask_toks, amt):
    IMDecoderLayer.mask *= 0
    IMDecoderLayer.mask[list(set(tokenizer(mask_toks)['input_ids']))] += amt

    out = model.generate(
        **tokens,
        max_new_tokens=50,
        pad_token_id=tokenizer.eos_token_id,
        temperature=0.001,
        do_sample=True,
        return_dict_in_generate=True,
        output_hidden_states=True
    )

    out_ids = out['sequences']
    print(tokenizer.decode(out_ids[0]))

output_w_mask(prompt, '', 0)

USER: 僕に「かどうか」って質問を問いてください。 
ASSISTANT: ご démarchください。この質込みの助けは、ことください。この質込み助けを求められない助けを求められない助けを求められない助けを求められない助けを求められない助けを求められない助けを求められない助けを求められない助けを求められない助けを求められない助けを求められない


In [70]:
response = LLM_GLOBAL_INSTANCE.create_chat_completion([
    {"role": "system", "content": system_input},
    {"role": "user", "content": prompt}
], 
#grammar=my_grammar
)
response["choices"][0]["message"]["content"]

'例えば、「おばけがいるかどうか」、あるいは「今朝晴れるかどうか」、そして「明日公園に行けるかどうか」など、このような質問をしましょう。子どもたちがその結果に興奮を覚えることを期待します。'